# DifferentialEquations.jl Workshop Exercise Solutions
### Chris Rackauckas

# Problem 1: Investigating Sources of Randomness and Uncertainty in a Biological System

## Part 1: Simulating the Oregonator ODE model

In [ ]:
using DifferentialEquations, Plots
function orego(du,u,p,t)
  s,q,w = p
  y1,y2,y3 = u
  du[1] = s*(y2+y1*(1-q*y1-y2))
  du[2] = (y3-(1+y1)*y2)/s
  du[3] = w*(y1-y3)
end
p = [77.27,8.375e-6,0.161]
prob = ODEProblem(orego,[1.0,2.0,3.0],(0.0,360.0),p)
sol = solve(prob)
plot(sol)

In [ ]:
plot(sol,vars=(1,2,3))

## Part 2: Investigating Stiffness

In [ ]:
using BenchmarkTools
prob = ODEProblem(orego,[1.0,2.0,3.0],(0.0,50.0),p)
@btime sol = solve(prob,Tsit5())

In [ ]:
@btime sol = solve(prob,Rodas5())

## (Optional) Part 3: Specifying Analytical Jacobians (I)

## (Optional) Part 4: Automatic Symbolicification and Analytical Jacobian Calculations

## Part 5: Adding stochasticity with stochastic differential equations

In [ ]:
function orego(du,u,p,t)
  s,q,w = p
  y1,y2,y3 = u
  du[1] = s*(y2+y1*(1-q*y1-y2))
  du[2] = (y3-(1+y1)*y2)/s
  du[3] = w*(y1-y3)
end
function g(du,u,p,t)
  du[1] = 0.1u[1]
  du[2] = 0.1u[2]
  du[3] = 0.1u[3]
end
p = [77.27,8.375e-6,0.161]
prob = SDEProblem(orego,g,[1.0,2.0,3.0],(0.0,30.0),p)
sol = solve(prob,SOSRI())
plot(sol)

In [ ]:
sol = solve(prob,ImplicitRKMil()); plot(sol)

In [ ]:
sol = solve(prob,ImplicitRKMil()); plot(sol)

## Part 6: Gillespie jump models of discrete stochasticity

## Part 7: Probabilistic Programming / Bayesian Parameter Estimation with DiffEqBayes.jl + Turing.jl (I)

The data was generated with:

In [ ]:
function orego(du,u,p,t)
  s,q,w = p
  y1,y2,y3 = u
  du[1] = s*(y2+y1*(1-q*y1-y2))
  du[2] = (y3-(1+y1)*y2)/s
  du[3] = w*(y1-y3)
end
p = [60.0,1e-5,0.2]
prob = ODEProblem(orego,[1.0,2.0,3.0],(0.0,30.0),p)
sol = solve(prob,Rodas5(),abstol=1/10^14,reltol=1/10^14)

## (Optional) Part 8: Using DiffEqBiological's Reaction Network DSL

# Problem 2: Fitting Hybrid Delay Pharmacokinetic Models with Automated Responses (B)

## Part 1: Defining an ODE with Predetermined Doses

In [ ]:
function onecompartment(du,u,p,t)
  Ka,Ke = p
  du[1] = -Ka*u[1]
  du[2] =  Ka*u[1] - Ke*u[2]
end
p = (Ka=2.268,Ke=0.07398)
prob = ODEProblem(onecompartment,[100.0,0.0],(0.0,90.0),p)

tstops = [24,48,72]
condition(u,t,integrator) = t ∈ tstops
affect!(integrator) = (integrator.u[1] += 100)
cb = DiscreteCallback(condition,affect!)
sol = solve(prob,Tsit5(),callback=cb,tstops=tstops)
plot(sol)

## Part 2: Adding Delays

In [ ]:
function onecompartment_delay(du,u,h,p,t)
  Ka,Ke,τ = p
  delayed_depot = h(p,t-τ)[1]
  du[1] = -Ka*u[1]
  du[2] =  Ka*delayed_depot - Ke*u[2]
end
p = (Ka=2.268,Ke=0.07398,τ=6.0)
h(p,t) = [0.0,0.0]
prob = DDEProblem(onecompartment_delay,[100.0,0.0],h,(0.0,90.0),p)

tstops = [24,48,72]
condition(u,t,integrator) = t ∈ tstops
affect!(integrator) = (integrator.u[1] += 100)
cb = DiscreteCallback(condition,affect!)
sol = solve(prob,MethodOfSteps(Rosenbrock23()),callback=cb,tstops=tstops)
plot(sol)

## Part 3: Automatic Differentiation (AD) for Optimization (I)

## Part 4: Fitting Known Quantities with DiffEqParamEstim.jl + Optim.jl

The data was generated with

In [ ]:
p = (Ka = 0.5, Ke = 0.1, τ = 4.0)

## Part 5: Implementing Control-Based Logic with ContinuousCallbacks (I)

## Part 6: Global Sensitivity Analysis with the Morris and Sobol Methods

# Problem 3: Differential-Algebraic Equation Modeling of a Double Pendulum (B)

## Part 1: Simple Introduction to DAEs: Mass-Matrix Robertson Equations

## Part 2: Solving the Implicit Robertson Equations with IDA

## Part 3: Manual Index Reduction of the Single Pendulum

## Part 4: Single Pendulum Solution with IDA

## Part 5: Solving the Double Penulum DAE System

# Problem 4: Performance Optimizing and Parallelizing Semilinear PDE Solvers (I)

## Part 1: Implementing the BRUSS PDE System as ODEs

## Part 2: Optimizing the BRUSS Code

## Part 3: Exploiting Jacobian Sparsity with Color Differentiation

## (Optional) Part 4: Structured Jacobians

## (Optional) Part 5: Automatic Symbolicification and Analytical Jacobian

## Part 6: Utilizing Preconditioned-GMRES Linear Solvers

## Part 7: Exploring IMEX and Exponential Integrator Techniques (E)

## Part 8: Work-Precision Diagrams for Benchmarking Solver Choices

## Part 9: GPU-Parallelism for PDEs (E)

## Part 10: Adjoint Sensitivity Analysis for Gradients of PDEs

# Problem 5: Global Parameter Sensitivity and Optimality with GPU and Distributed Ensembles (B)

## Part 1: Implementing the Henon-Heiles System (B)

## (Optional) Part 2: Alternative Dynamical Implmentations of Henon-Heiles (B)

## Part 3: Parallelized Ensemble Solving

## Part 4: Parallelized GPU Ensemble Solving

# Problem 6: Training Neural Stochastic Differential Equations with GPU acceleration (I)

## Part 1: Constructing and Training a Basic Neural ODE

## Part 2: GPU-accelerating the Neural ODE Process

## Part 3: Defining and Training a Mixed Neural ODE

## Part 4: Constructing a Basic Neural SDE

## Part 5: Optimizing the training behavior with minibatching (E)